In [1]:
!pip install flask langchain beautifulsoup4 requests python-dotenv flask-cors

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install langchain-community
!pip install --upgrade langchain

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
pip install utils

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
%%writefile .env
SERPAPI_KEY=3ad2c2af46b15c2abfc889813f9ec37efb44b2ecf4439dc5bb1438e37f270c48
OPENAI_API_KEY=sk-mg7j85OWtVxUVFi2PyM7yzRatBL8D-fDEkWZ0E4F8YT3BlbkFJHuiZirtYrikFftp4sgO_l-ueOHzHvTISQAibasLJUA


Overwriting .env


In [5]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import os
os.makedirs("flask_app", exist_ok=True)

In [1]:
from flask import jsonify

In [1]:
%%writefile flask_app/app.py
from flask import Flask, jsonify, request
from flask_cors import CORS
import requests
from bs4 import BeautifulSoup
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
import os
from dotenv import load_dotenv


load_dotenv()
app = Flask(__name__)
CORS(app)

memory = ConversationBufferMemory()
llm = OpenAI(temperature=0.7)

def get_wikipedia_content(query):
    """
    Simple Wikipedia scraper - gets first 5 paragraphs
    Works like Google search but for Wikipedia
    """
    try:
        # 1. Make Wikipedia URL
        url = f"https://en.wikipedia.org/wiki/{query.replace(' ', '_')}"
        
        # 2. Get page content
        response = requests.get(url)
        if response.status_code != 200:
            return "Couldn't find Wikipedia page"
        
        # 3. Soup is like a smart text filter
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 4. Find main content area
        content = soup.find('div', {'id': 'mw-content-text'})
        
        # 5. Get first 5 paragraphs (like book pages)
        paragraphs = [p.get_text() for p in content.find_all('p')[:5]]
        
        return ' '.join(paragraphs)
    
    except Exception as e:
        return f"Error: {str(e)}"

# Change the route to handle AI processing
@app.route('/query', methods=['POST'])
def handle_query():
    """
    This is like our AI factory worker
    """
    data = request.json
    user_question = data['query']
    
    # 1. Get Wikipedia info
    context = get_wikipedia_content(user_question)
    
    # 2. Prepare AI worker with memory
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=lambda _: [context],  # Our Wikipedia info
        memory=memory
    )
    
    # 3. Generate answer
    answer = qa.run(user_question)
    
    return jsonify({'response': answer})

if __name__ == '__main__':
    app.run(port=5000)


Overwriting flask_app/app.py


In [ ]:
!cd flask_app && python app.py